In [ ]:
import joblib

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor

In [ ]:
df = pd.read_csv('data3_0505.csv', index_col=0)

In [ ]:
df.info()

In [ ]:
X=df.drop(['price'], axis=1)
y = df['price']

### Training/Test Set Split and MinMaxScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7633)

In [ ]:
print(X_train.shape)

(16209, 29)


In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler()

In [ ]:
# X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index)
# X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index)

#### Load Best Models

In [ ]:
best_xgb = joblib.load('best_xgb_log_0506_9195.joblib')
best_stack = joblib.load('best_stack_0507_9198+1069.joblib')

/Users/chongqi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MLPRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/chongqi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVR from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/chongqi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Ridge from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results.

### 'view'

In [ ]:
# Group by 'view' and calculate the mean of 'price'
df_view = df.groupby('view')['price'].mean().reset_index()

In [ ]:
# Rename columns
df_view.columns = ['view', 'average_price']

In [ ]:
# add column 'count'
df_view['count'] = df['view'].value_counts().sort_index().values

In [ ]:
df_view.head()

,view,average_price,count
0,0,4.965642e+05,19489
1,1,8.122808e+05,332
2,2,7.924009e+05,963
3,3,9.719653e+05,510
4,4,1.463711e+06,319


In [ ]:
# create empty dictionary to store indexes
index_dict = {}

# loop through view levels 0 to 4
for view_level in range(5):
    # get indexes where 'view' is equal to current level
    index_dict[f"index_{view_level}"] = df[df['view'] == view_level].index.tolist()



In [ ]:
# set the value of 'view' to 0, 1, 2, 3, 4, respectively
for view_level in range(5):
    X_copy = df.drop(['price'], axis=1).copy()
    X_copy['view'] = view_level

    # go through the same preprocessing steps
    X_copy = pd.DataFrame(scaler.transform(X_copy), index=X_copy.index)

    # predict prices using the trained model
    predicted_prices = pd.DataFrame(np.exp(best_xgb.predict(X_copy)), index=X_copy.index)

    avg_pred_price = {}
    for j in range(5):
        # get the indexes for the current view level
        index_list = index_dict[f"index_{j}"]

        # calculate the average predicted price
        avg_pred_price[j] = np.average(predicted_prices.loc[index_list])

    print(avg_pred_price)

    # update df_view with the average predicted price
    df_view[f"aver_pred_price_{view_level}"] = df_view['view'].map(avg_pred_price)



{0: 491545.38, 1: 741740.5, 2: 726422.0, 3: 830254.8, 4: 1101354.9}
{0: 533787.44, 1: 799857.25, 2: 780477.7, 3: 893511.25, 4: 1201899.4}
{0: 533916.6, 1: 804613.06, 2: 783176.7, 3: 896722.44, 4: 1207828.0}
{0: 558542.56, 1: 845224.1, 2: 820767.4, 3: 940101.5, 4: 1267892.1}
{0: 618901.0, 1: 943943.06, 2: 914085.4, 3: 1044460.3, 4: 1419771.4}


In [ ]:
df_view.head()

,view,average_price,count,aver_pred_price_0,aver_pred_price_1,aver_pred_price_2,aver_pred_price_3,aver_pred_price_4
0,0,4.965642e+05,19489,4.915454e+05,5.337874e+05,5.339166e+05,5.585426e+05,6.189010e+05
1,1,8.122808e+05,332,7.417405e+05,7.998572e+05,8.046131e+05,8.452241e+05,9.439431e+05
2,2,7.924009e+05,963,7.264220e+05,7.804777e+05,7.831767e+05,8.207674e+05,9.140854e+05
3,3,9.719653e+05,510,8.302548e+05,8.935112e+05,8.967224e+05,9.401015e+05,1.044460e+06
4,4,1.463711e+06,319,1.101355e+06,1.201899e+06,1.207828e+06,1.267892e+06,1.419771e+06


In [ ]:
df_view.to_csv('view.csv', index=False)

In [ ]:
# causal inference of 'view' with stacking model

# Group by 'view' and calculate the mean of 'price'
df_view_stack = df.groupby('view')['price'].mean().reset_index()

# Rename columns
df_view_stack.columns = ['view', 'average_price']

# add column 'count'
df_view_stack['count'] = df['view'].value_counts().sort_index().values

# create empty dictionary to store indexes
index_dict = {}

# loop through view levels 0 to 4
for view_level in range(5):
    # get indexes where 'view' is equal to current level
    index_dict[f"index_{view_level}"] = df[df['view'] == view_level].index.tolist()

# set the value of 'view' to 0, 1, 2, 3, 4, respectively
for view_level in range(5):
    X_copy = df.drop(['price'], axis=1).copy()
    X_copy['view'] = view_level

    # go through the same preprocessing steps
    X_copy = pd.DataFrame(scaler.transform(X_copy), index=X_copy.index)

    # predict prices using the trained model
    predicted_prices = pd.DataFrame(np.exp(best_stack.predict(X_copy)), index=X_copy.index)

    avg_pred_price = {}
    for j in range(5):
        # get the indexes for the current view level
        index_list = index_dict[f"index_{j}"]

        # calculate the average predicted price
        avg_pred_price[j] = np.average(predicted_prices.loc[index_list])

    print(avg_pred_price)

    # update df_view with the average predicted price
    df_view_stack[f"aver_pred_price_{view_level}"] = df_view['view'].map(avg_pred_price)



{0: 491658.1933488775, 1: 741566.7596768569, 2: 726772.2849281663, 3: 831559.1293129807, 4: 1139154.1487138811}
{0: 529459.8599198346, 1: 795651.5464261597, 2: 776306.0758987784, 3: 888854.598298626, 4: 1225134.3336484141}
{0: 534448.574778659, 1: 808451.4437013569, 2: 786107.1764131808, 3: 899673.7602672789, 4: 1236200.522347677}
{0: 559469.05337063, 1: 851174.0733495136, 2: 825096.6891688677, 3: 944004.1248770942, 4: 1293196.1852944547}
{0: 615552.7607150384, 1: 944748.9052152439, 2: 912859.8677447244, 3: 1041830.0493407209, 4: 1430388.4996637322}


In [ ]:
df_view_stack.head()

,view,average_price,count,aver_pred_price_0,aver_pred_price_1,aver_pred_price_2,aver_pred_price_3,aver_pred_price_4
0,0,4.965642e+05,19489,4.916582e+05,5.294599e+05,5.344486e+05,5.594691e+05,6.155528e+05
1,1,8.122808e+05,332,7.415668e+05,7.956515e+05,8.084514e+05,8.511741e+05,9.447489e+05
2,2,7.924009e+05,963,7.267723e+05,7.763061e+05,7.861072e+05,8.250967e+05,9.128599e+05
3,3,9.719653e+05,510,8.315591e+05,8.888546e+05,8.996738e+05,9.440041e+05,1.041830e+06
4,4,1.463711e+06,319,1.139154e+06,1.225134e+06,1.236201e+06,1.293196e+06,1.430388e+06


In [ ]:
# causal inference of 'waterfront' with xgb model

# Group by 'waterfront' and calculate the mean of 'price'
df_waterfront = df.groupby('waterfront')['price'].mean().reset_index()

# Rename columns
df_waterfront.columns = ['waterfront', 'average_price']

# add column 'count'
df_waterfront['count'] = df['waterfront'].value_counts().sort_index().values

# create empty dictionary to store indexes
index_dict = {}

# Get the unique values
unique_values = df['waterfront'].unique()

# loop through all unique values
for level in unique_values:
    # get indexes where the value is equal to current level
    index_dict[f"index_{level}"] = df[df['waterfront'] == level].index.tolist()

# set the value to each unique value, respectively
for level in unique_values:
    X_copy = df.drop(['price'], axis=1).copy()
    X_copy['waterfront'] = level

    # go through the same preprocessing steps
    X_copy = pd.DataFrame(scaler.transform(X_copy), index=X_copy.index)

    # predict prices using the trained model
    predicted_prices = pd.DataFrame(np.exp(best_xgb.predict(X_copy)), index=X_copy.index)

    avg_pred_price = {}
    for j in unique_values:
        # get the indexes for the current level
        index_list = index_dict[f"index_{j}"]

        # calculate the average predicted price
        avg_pred_price[j] = np.average(predicted_prices.loc[index_list])

    print(avg_pred_price)

    # update dataframe with the average predicted price
    df_waterfront[f"aver_pred_price_{level}"] = df_waterfront['waterfront'].map(avg_pred_price)



{0: 525452.2, 1: 1085601.0}
{0: 751280.1, 1: 1600550.1}


In [ ]:
df_waterfront.head()

,waterfront,average_price,count,aver_pred_price_0,aver_pred_price_1
0,0,5.315636e+05,21450,5.254522e+05,751280.125
1,1,1.661876e+06,163,1.085601e+06,1600550.125


In [ ]:
df_waterfront.to_csv('waterfront.csv', index=False)

In [ ]:
# causal inference of 'condition' with xgb model

# Group by 'condition' and calculate the mean of 'price'
df_condition = df.groupby('condition')['price'].mean().reset_index()

# Rename columns
df_condition.columns = ['condition', 'average_price']

# add column 'count'
df_condition['count'] = df['condition'].value_counts().sort_index().values

# create empty dictionary to store indexes
index_dict = {}

# Get the unique values in the 'grade' column
unique_values = df['condition'].unique()

# Sort the unique values in ascending order
unique_values_sorted = sorted(unique_values)

# loop through all unique values
for level in unique_values_sorted:
    # get indexes where the value is equal to current level
    index_dict[f"index_{level}"] = df[df['condition'] == level].index.tolist()

# set the value to each unique value, respectively
for level in unique_values_sorted:
    X_copy = df.drop(['price'], axis=1).copy()
    X_copy['condition'] = level

    # go through the same preprocessing steps
    X_copy = pd.DataFrame(scaler.transform(X_copy), index=X_copy.index)

    # predict prices using the trained model
    predicted_prices = pd.DataFrame(np.exp(best_xgb.predict(X_copy)), index=X_copy.index)

    avg_pred_price = {}
    for j in unique_values_sorted:
        # get the indexes for the current level
        index_list = index_dict[f"index_{j}"]

        # calculate the average predicted price
        avg_pred_price[j] = np.average(predicted_prices.loc[index_list])

    print(avg_pred_price)

    # update dataframe with the average predicted price
    df_condition[f"aver_pred_price_{level}"] = df_condition['condition'].map(avg_pred_price)



{1: 305871.16, 2: 288459.16, 3: 442286.66, 4: 399629.47, 5: 436463.75}
{1: 329439.56, 2: 313935.7, 3: 483045.5, 4: 437206.72, 5: 479538.12}
{1: 375874.38, 2: 351845.7, 3: 535556.1, 4: 487908.78, 5: 537397.94}
{1: 404218.66, 2: 375312.0, 3: 555600.25, 4: 514312.06, 5: 567699.8}
{1: 428238.12, 2: 396163.06, 3: 581361.56, 4: 548767.4, 5: 607583.4}


In [ ]:
df_condition.head()

,condition,average_price,count,aver_pred_price_1,aver_pred_price_2,aver_pred_price_3,aver_pred_price_4,aver_pred_price_5
0,1,334431.666667,30,305871.15625,329439.56250,375874.37500,404218.65625,428238.1250
1,2,327287.145349,172,288459.15625,313935.68750,351845.68750,375312.00000,396163.0625
2,3,542012.578148,14031,442286.65625,483045.50000,535556.12500,555600.25000,581361.5625
3,4,521200.390033,5679,399629.46875,437206.71875,487908.78125,514312.06250,548767.3750
4,5,612418.089359,1701,436463.75000,479538.12500,537397.93750,567699.81250,607583.3750


In [ ]:
df_condition.to_csv('condition.csv', index=False)

In [ ]:
# causal inference of 'grade' with xgb model

# Group by 'grade' and calculate the mean of 'price'
df_grade = df.groupby('grade')['price'].mean().reset_index()

# Rename columns
df_grade.columns = ['grade', 'average_price']

# add column 'count'
df_grade['count'] = df['grade'].value_counts().sort_index().values

# create empty dictionary to store indexes
index_dict = {}

# Get the unique values in the 'grade' column
unique_values = df['grade'].unique()

# Sort the unique values in ascending order
unique_values_sorted = sorted(unique_values)

# loop through all unique values
for level in unique_values_sorted:
    # get indexes where the value is equal to current level
    index_dict[f"index_{level}"] = df[df['grade'] == level].index.tolist()

# set the value to each unique value, respectively
for level in unique_values_sorted:
    X_copy = df.drop(['price'], axis=1).copy()
    X_copy['grade'] = level

    # go through the same preprocessing steps
    X_copy = pd.DataFrame(scaler.transform(X_copy), index=X_copy.index)

    # predict prices using the trained model
    predicted_prices = pd.DataFrame(np.exp(best_xgb.predict(X_copy)), index=X_copy.index)

    avg_pred_price = {}
    for j in unique_values_sorted:
        # get the indexes for the current level
        index_list = index_dict[f"index_{j}"]

        # calculate the average predicted price
        avg_pred_price[j] = np.average(predicted_prices.loc[index_list])

    print(avg_pred_price)

    # update dataframe with the average predicted price
    df_grade[f"aver_pred_price_{level}"] = df_grade['grade'].map(avg_pred_price)

df_grade.head(15)

{1: 142931.95, 3: 142794.33, 4: 183258.2, 5: 195434.19, 6: 230627.84, 7: 300899.16, 8: 383713.62, 9: 497054.16, 10: 618541.06, 11: 793055.25, 12: 1024413.9, 13: 1346384.0}
{1: 161414.47, 3: 156452.86, 4: 201079.67, 5: 213277.03, 6: 253168.64, 7: 326175.4, 8: 411060.28, 9: 530681.25, 10: 660003.9, 11: 846093.9, 12: 1092925.4, 13: 1436428.5}
{1: 181391.12, 3: 185833.94, 4: 217222.44, 5: 230053.25, 6: 266616.06, 7: 341624.03, 8: 429960.56, 9: 555472.4, 10: 686588.94, 11: 871252.56, 12: 1121007.9, 13: 1470234.8}
{1: 180105.72, 3: 187075.55, 4: 222880.23, 5: 236592.1, 6: 275499.66, 7: 354562.03, 8: 445780.53, 9: 574700.7, 10: 712269.56, 11: 894035.25, 12: 1145314.5, 13: 1490004.2}
{1: 186540.73, 3: 191587.58, 4: 236418.95, 5: 252291.88, 6: 297147.72, 7: 371472.53, 8: 462380.62, 9: 592731.56, 10: 733224.94, 11: 920884.6, 12: 1187466.6, 13: 1530049.1}
{1: 193943.55, 3: 201770.8, 4: 253002.8, 5: 271254.12, 6: 317890.84, 7: 397918.2, 8: 495503.53, 9: 640728.8, 10: 801478.75, 11: 1003239.9, 12: 

,grade,average_price,count,aver_pred_price_1,aver_pred_price_3,aver_pred_price_4,aver_pred_price_5,aver_pred_price_6,aver_pred_price_7,aver_pred_price_8,aver_pred_price_9,aver_pred_price_10,aver_pred_price_11,aver_pred_price_12,aver_pred_price_13
0,1,1.420000e+05,1,1.429320e+05,1.614145e+05,1.813911e+05,1.801057e+05,1.865407e+05,1.939435e+05,2.093950e+05,2.404415e+05,2.612565e+05,2.992234e+05,3.278566e+05,3.278566e+05
1,3,2.056667e+05,3,1.427943e+05,1.564529e+05,1.858339e+05,1.870755e+05,1.915876e+05,2.017708e+05,2.179154e+05,2.444565e+05,2.665126e+05,2.979350e+05,3.214535e+05,3.214535e+05
2,4,2.143810e+05,29,1.832582e+05,2.010797e+05,2.172224e+05,2.228802e+05,2.364190e+05,2.530028e+05,2.758300e+05,3.045937e+05,3.326868e+05,3.776416e+05,4.130187e+05,4.131160e+05
3,5,2.485240e+05,242,1.954342e+05,2.132770e+05,2.300532e+05,2.365921e+05,2.522919e+05,2.712541e+05,2.975718e+05,3.280284e+05,3.590092e+05,4.029028e+05,4.379099e+05,4.379675e+05
4,6,3.019196e+05,2038,2.306278e+05,2.531686e+05,2.666161e+05,2.754997e+05,2.971477e+05,3.178908e+05,3.483519e+05,3.870235e+05,4.269523e+05,4.782108e+05,5.202338e+05,5.203836e+05
5,7,4.025903e+05,8981,3.008992e+05,3.261754e+05,3.416240e+05,3.545620e+05,3.714725e+05,3.979182e+05,4.316821e+05,4.787075e+05,5.276321e+05,5.877909e+05,6.420046e+05,6.423856e+05
6,8,5.428528e+05,6068,3.837136e+05,4.110603e+05,4.299606e+05,4.457805e+05,4.623806e+05,4.955035e+05,5.369624e+05,5.913561e+05,6.501257e+05,7.209505e+05,7.894328e+05,7.901281e+05
7,9,7.735132e+05,2615,4.970542e+05,5.306812e+05,5.554724e+05,5.747007e+05,5.927316e+05,6.407288e+05,6.972592e+05,7.663918e+05,8.441091e+05,9.315508e+05,1.024354e+06,1.025695e+06
8,10,1.071771e+06,1134,6.185411e+05,6.600039e+05,6.865889e+05,7.122696e+05,7.332249e+05,8.014788e+05,8.715480e+05,9.635589e+05,1.064071e+06,1.170218e+06,1.292821e+06,1.295596e+06
9,11,1.496842e+06,399,7.930552e+05,8.460939e+05,8.712526e+05,8.940352e+05,9.208846e+05,1.003240e+06,1.088362e+06,1.211067e+06,1.339384e+06,1.478104e+06,1.637609e+06,1.642493e+06


In [ ]:
df_grade.to_csv('grade.csv', index=False)

In [ ]:
# causal inference of 'bedrooms' with xgb model

# Group by 'bedrooms' and calculate the mean of 'price'
df_bedrooms = df.groupby('bedrooms')['price'].mean().reset_index()

# Rename columns
df_bedrooms.columns = ['bedrooms', 'average_price']

# add column 'count'
df_bedrooms['count'] = df['bedrooms'].value_counts().sort_index().values

# create empty dictionary to store indexes
index_dict = {}

# Get the unique values in the 'bedrooms' column
unique_values = df['bedrooms'].unique()

# Sort the unique values in ascending order
unique_values_sorted = sorted(unique_values)

# loop through all unique values
for level in unique_values_sorted:
    # get indexes where the value is equal to current level
    index_dict[f"index_{level}"] = df[df['bedrooms'] == level].index.tolist()

# set the value to each unique value, respectively
for level in unique_values_sorted:
    X_copy = df.drop(['price'], axis=1).copy()
    X_copy['bedrooms'] = level

    # go through the same preprocessing steps
    X_copy = pd.DataFrame(scaler.transform(X_copy), index=X_copy.index)

    # predict prices using the trained model
    predicted_prices = pd.DataFrame(np.exp(best_xgb.predict(X_copy)), index=X_copy.index)

    avg_pred_price = {}
    for j in unique_values_sorted:
        # get the indexes for the current level
        index_list = index_dict[f"index_{j}"]

        # calculate the average predicted price
        avg_pred_price[j] = np.average(predicted_prices.loc[index_list])

    print(avg_pred_price)

    # update dataframe with the average predicted price
    df_bedrooms[f"aver_pred_price_{level}"] = df_bedrooms['bedrooms'].map(avg_pred_price)

df_bedrooms.head(15)

{0: 394849.06, 1: 305706.22, 2: 388429.16, 3: 453535.7, 4: 623078.3, 5: 772953.9, 6: 800934.7, 7: 950835.06, 8: 1016051.25, 9: 971486.7, 10: 942236.25, 11: 503179.94}
{0: 396601.28, 1: 305738.56, 2: 388478.72, 3: 453588.97, 4: 623351.56, 5: 773855.25, 6: 802150.94, 7: 951655.06, 8: 1019993.0, 9: 971086.2, 10: 942776.5, 11: 503179.94}
{0: 401464.16, 1: 309839.2, 2: 394633.56, 3: 460429.6, 4: 631753.4, 5: 784102.75, 6: 813216.25, 7: 964869.5, 8: 1036882.6, 9: 993675.2, 10: 960724.75, 11: 503377.22}
{0: 401424.97, 1: 310427.1, 2: 395198.28, 3: 460579.47, 4: 629675.44, 5: 781409.44, 6: 811435.44, 7: 962071.9, 8: 1035640.7, 9: 991856.0, 10: 953102.7, 11: 497550.2}
{0: 402455.7, 1: 310562.47, 2: 394518.53, 3: 460407.38, 4: 629978.56, 5: 782368.6, 6: 812492.7, 7: 963676.1, 8: 1038496.6, 9: 987741.2, 10: 952556.3, 11: 495178.16}
{0: 400443.97, 1: 309867.94, 2: 392942.38, 3: 457432.53, 4: 625429.94, 5: 776762.7, 6: 806171.75, 7: 958382.8, 8: 1030661.9, 9: 981124.8, 10: 944835.56, 11: 486247.88}

,bedrooms,average_price,count,aver_pred_price_0,aver_pred_price_1,aver_pred_price_2,aver_pred_price_3,aver_pred_price_4,aver_pred_price_5,aver_pred_price_6,aver_pred_price_7,aver_pred_price_8,aver_pred_price_9,aver_pred_price_10,aver_pred_price_11
0,0,4.095038e+05,13,3.948491e+05,3.966013e+05,4.014642e+05,4.014250e+05,4.024557e+05,4.004440e+05,3.938955e+05,3.934380e+05,3.934380e+05,388227.03125,388227.03125,388227.03125
1,1,3.176429e+05,199,3.057062e+05,3.057386e+05,3.098392e+05,3.104271e+05,3.105625e+05,3.098679e+05,3.063504e+05,3.057592e+05,3.057486e+05,305621.65625,305621.65625,305621.65625
2,2,4.013727e+05,2760,3.884292e+05,3.884787e+05,3.946336e+05,3.951983e+05,3.945185e+05,3.929424e+05,3.883710e+05,3.875200e+05,3.874080e+05,387263.15625,387263.15625,387263.15625
3,3,4.662498e+05,9825,4.535357e+05,4.535890e+05,4.604296e+05,4.605795e+05,4.604074e+05,4.574325e+05,4.506493e+05,4.494978e+05,4.492370e+05,448504.34375,448504.34375,448504.34375
4,4,6.354195e+05,6882,6.230783e+05,6.233516e+05,6.317534e+05,6.296754e+05,6.299786e+05,6.254299e+05,6.147415e+05,6.124242e+05,6.118968e+05,607321.25000,607321.25000,607321.25000
5,5,7.865998e+05,1601,7.729539e+05,7.738552e+05,7.841028e+05,7.814094e+05,7.823686e+05,7.767627e+05,7.626623e+05,7.579814e+05,7.569809e+05,747226.00000,747226.00000,747226.00000
6,6,8.255206e+05,272,8.009347e+05,8.021509e+05,8.132162e+05,8.114354e+05,8.124927e+05,8.061718e+05,7.907635e+05,7.850984e+05,7.846165e+05,775303.81250,775303.81250,775303.81250
7,7,9.511847e+05,38,9.508351e+05,9.516551e+05,9.648695e+05,9.620719e+05,9.636761e+05,9.583828e+05,9.410669e+05,9.366022e+05,9.359159e+05,913788.43750,913788.43750,913788.43750
8,8,1.105077e+06,13,1.016051e+06,1.019993e+06,1.036883e+06,1.035641e+06,1.038497e+06,1.030662e+06,1.011299e+06,1.007342e+06,1.006807e+06,982825.75000,982825.75000,982825.75000
9,9,8.939998e+05,6,9.714867e+05,9.710862e+05,9.936752e+05,9.918560e+05,9.877412e+05,9.811248e+05,9.544068e+05,9.451717e+05,9.401288e+05,913281.68750,913281.68750,913281.68750


In [ ]:
df_bedrooms.to_csv('bedrooms.csv', index=False)

In [ ]:
# causal inference of 'bathrooms' with xgb model

# Group by 'bathrooms' and calculate the mean of 'price'
df_bathrooms = df.groupby('bathrooms')['price'].mean().reset_index()

# Rename columns
df_bathrooms.columns = ['bathrooms', 'average_price']

# add column 'count'
df_bathrooms['count'] = df['bathrooms'].value_counts().sort_index().values

# create empty dictionary to store indexes
index_dict = {}

# Get the unique values in the 'bathrooms' column
unique_values = df['bathrooms'].unique()

# Sort the unique values in ascending order
unique_values_sorted = sorted(unique_values)

# loop through all unique values
for level in unique_values_sorted:
    # get indexes where the value is equal to current level
    index_dict[f"index_{level}"] = df[df['bathrooms'] == level].index.tolist()

# set the value to each unique value, respectively
for level in unique_values_sorted:
    X_copy = df.drop(['price'], axis=1).copy()
    X_copy['bathrooms'] = level

    # go through the same preprocessing steps
    X_copy = pd.DataFrame(scaler.transform(X_copy), index=X_copy.index)

    # predict prices using the trained model
    predicted_prices = pd.DataFrame(np.exp(best_xgb.predict(X_copy)), index=X_copy.index)

    avg_pred_price = {}
    for j in unique_values_sorted:
        # get the indexes for the current level
        index_list = index_dict[f"index_{j}"]

        # calculate the average predicted price
        avg_pred_price[j] = np.average(predicted_prices.loc[index_list])

    print(avg_pred_price)

    # update dataframe with the average predicted price
    df_bathrooms[f"aver_pred_price_{level}"] = df_bathrooms['bathrooms'].map(avg_pred_price)

df_bathrooms.head(30)

{0.0: 411856.12, 0.5: 227293.9, 0.75: 271198.9, 1.0: 317495.5, 1.25: 593318.56, 1.5: 375789.94, 1.75: 415295.47, 2.0: 418187.5, 2.25: 482950.0, 2.5: 502929.22, 2.75: 604774.3, 3.0: 643248.44, 3.25: 869654.6, 3.5: 853394.94, 3.75: 1099958.1, 4.0: 1130032.0, 4.25: 1368026.0, 4.5: 1179399.4, 4.75: 1791681.0, 5.0: 1376030.9, 5.25: 1692680.0, 5.5: 2060253.8, 5.75: 1959180.0, 6.0: 2363617.0, 6.25: 2354298.0, 6.5: 1455975.1, 6.75: 2470018.5, 7.5: 589988.3, 7.75: 4748506.0, 8.0: 2272541.5}
{0.0: 419619.94, 0.5: 238169.06, 0.75: 285051.72, 1.0: 332150.03, 1.25: 597720.6, 1.5: 385863.44, 1.75: 423279.3, 2.0: 426774.1, 2.25: 489322.7, 2.5: 509867.34, 2.75: 611614.1, 3.0: 650242.44, 3.25: 874349.94, 3.5: 858776.9, 3.75: 1104703.4, 4.0: 1134192.1, 4.25: 1372354.4, 4.5: 1183968.8, 4.75: 1793927.6, 5.0: 1381204.2, 5.25: 1693493.1, 5.5: 2068230.4, 5.75: 1959180.0, 6.0: 2363617.0, 6.25: 2354298.0, 6.5: 1455975.1, 6.75: 2470018.5, 7.5: 589988.3, 7.75: 4748506.0, 8.0: 2272541.5}
{0.0: 419957.0, 0.5: 2398

{0.0: 455261.84, 0.5: 254770.53, 0.75: 308335.75, 1.0: 356124.3, 1.25: 630674.44, 1.5: 415569.53, 1.75: 459719.3, 2.0: 461696.9, 2.25: 533798.1, 2.5: 561577.0, 2.75: 667991.6, 3.0: 703822.25, 3.25: 953608.94, 3.5: 942951.5, 3.75: 1206830.9, 4.0: 1250425.5, 4.25: 1507979.5, 4.5: 1306630.1, 4.75: 1996113.8, 5.0: 1550869.8, 5.25: 1910588.1, 5.5: 2356934.0, 5.75: 2272840.5, 6.0: 2648763.2, 6.25: 2840176.5, 6.5: 1667231.5, 6.75: 2738697.8, 7.5: 598315.9, 7.75: 5499429.5, 8.0: 2637447.0}
{0.0: 451690.94, 0.5: 254770.53, 0.75: 308335.75, 1.0: 356108.7, 1.25: 630674.44, 1.5: 415448.7, 1.75: 459577.7, 2.0: 461560.03, 2.25: 533211.4, 2.5: 559509.06, 2.75: 665072.0, 3.0: 701884.6, 3.25: 948262.44, 3.5: 936869.5, 3.75: 1200763.9, 4.0: 1240926.5, 4.25: 1498347.9, 4.5: 1298142.0, 4.75: 1986322.2, 5.0: 1540144.5, 5.25: 1902625.4, 5.5: 2350569.0, 5.75: 2272840.5, 6.0: 2644776.2, 6.25: 2840176.5, 6.5: 1666081.0, 6.75: 2738697.8, 7.5: 598315.9, 7.75: 5499429.5, 8.0: 2612444.5}
{0.0: 449308.4, 0.5: 25296

,bathrooms,average_price,count,aver_pred_price_0.0,aver_pred_price_0.5,aver_pred_price_0.75,aver_pred_price_1.0,aver_pred_price_1.25,aver_pred_price_1.5,aver_pred_price_1.75,...,aver_pred_price_5.25,aver_pred_price_5.5,aver_pred_price_5.75,aver_pred_price_6.0,aver_pred_price_6.25,aver_pred_price_6.5,aver_pred_price_6.75,aver_pred_price_7.5,aver_pred_price_7.75,aver_pred_price_8.0
0,0.00,4.481600e+05,10,4.118561e+05,4.196199e+05,4.199570e+05,4.201192e+05,4.236171e+05,4.244289e+05,4.348688e+05,...,4.552618e+05,4.516909e+05,4.493084e+05,4.493084e+05,4.486231e+05,4.475559e+05,4.446382e+05,4.446382e+05,4.446382e+05,4.446382e+05
1,0.50,2.373750e+05,4,2.272939e+05,2.381691e+05,2.398444e+05,2.393769e+05,2.413234e+05,2.420803e+05,2.486066e+05,...,2.547705e+05,2.547705e+05,2.529651e+05,2.529651e+05,2.504642e+05,2.504642e+05,2.504642e+05,2.502989e+05,2.502989e+05,2.502989e+05
2,0.75,2.945209e+05,72,2.711989e+05,2.850517e+05,2.865907e+05,2.900602e+05,2.937693e+05,2.947349e+05,3.010162e+05,...,3.083358e+05,3.083358e+05,3.080128e+05,3.080128e+05,3.074007e+05,3.063036e+05,3.063036e+05,3.063036e+05,3.063036e+05,3.063036e+05
3,1.00,3.470412e+05,3852,3.174955e+05,3.321500e+05,3.331547e+05,3.411920e+05,3.451860e+05,3.458339e+05,3.517227e+05,...,3.561243e+05,3.561087e+05,3.553532e+05,3.553532e+05,3.542671e+05,3.523878e+05,3.523878e+05,3.523873e+05,3.523873e+05,3.523873e+05
4,1.25,6.212167e+05,9,5.933186e+05,5.977206e+05,5.976899e+05,6.087624e+05,6.119989e+05,6.119779e+05,6.177818e+05,...,6.306744e+05,6.306744e+05,6.295281e+05,6.295281e+05,6.296668e+05,6.207758e+05,6.163704e+05,6.161056e+05,6.161056e+05,6.161056e+05
5,1.50,4.093222e+05,1446,3.757899e+05,3.858634e+05,3.868681e+05,3.975129e+05,4.015581e+05,4.020533e+05,4.085005e+05,...,4.155695e+05,4.154487e+05,4.134106e+05,4.134106e+05,4.118497e+05,4.076628e+05,4.076438e+05,4.076430e+05,4.076430e+05,4.076430e+05
6,1.75,4.548961e+05,3048,4.152955e+05,4.232793e+05,4.240397e+05,4.378610e+05,4.423110e+05,4.428472e+05,4.501202e+05,...,4.597193e+05,4.595777e+05,4.555397e+05,4.555384e+05,4.540280e+05,4.496765e+05,4.496632e+05,4.496601e+05,4.496601e+05,4.496601e+05
7,2.00,4.578897e+05,1930,4.181875e+05,4.267741e+05,4.278942e+05,4.411425e+05,4.455492e+05,4.460228e+05,4.539864e+05,...,4.616969e+05,4.615600e+05,4.576765e+05,4.576762e+05,4.561931e+05,4.520255e+05,4.520255e+05,4.520039e+05,4.520039e+05,4.520039e+05
8,2.25,5.336768e+05,2047,4.829500e+05,4.893227e+05,4.898958e+05,5.031879e+05,5.072193e+05,5.076428e+05,5.180059e+05,...,5.337981e+05,5.332114e+05,5.239530e+05,5.239512e+05,5.224913e+05,5.154201e+05,5.153154e+05,5.152867e+05,5.152867e+05,5.152867e+05
9,2.50,5.535965e+05,5380,5.029292e+05,5.098673e+05,5.104518e+05,5.219698e+05,5.263586e+05,5.267058e+05,5.420598e+05,...,5.615770e+05,5.595091e+05,5.491553e+05,5.491515e+05,5.475312e+05,5.388321e+05,5.384398e+05,5.383403e+05,5.383403e+05,5.383403e+05


In [ ]:
df_bathrooms.to_csv('bathrooms.csv', index=False)

In [ ]:
# causal inference of 'floors' with xgb model

# Group by 'floors' and calculate the mean of 'price'
df_floors = df.groupby('floors')['price'].mean().reset_index()

# Rename columns
df_floors.columns = ['floors', 'average_price']

# add column 'count'
df_floors['count'] = df['floors'].value_counts().sort_index().values

# create empty dictionary to store indexes
index_dict = {}

# Get the unique values in the 'bathrooms' column
unique_values = df['floors'].unique()

# Sort the unique values in ascending order
unique_values_sorted = sorted(unique_values)

# loop through all unique values
for level in unique_values_sorted:
    # get indexes where the value is equal to current level
    index_dict[f"index_{level}"] = df[df['floors'] == level].index.tolist()

# set the value to each unique value, respectively
for level in unique_values_sorted:
    X_copy = df.drop(['price'], axis=1).copy()
    X_copy['floors'] = level

    # go through the same preprocessing steps
    X_copy = pd.DataFrame(scaler.transform(X_copy), index=X_copy.index)

    # predict prices using the trained model
    predicted_prices = pd.DataFrame(np.exp(best_xgb.predict(X_copy)), index=X_copy.index)

    avg_pred_price = {}
    for j in unique_values_sorted:
        # get the indexes for the current level
        index_list = index_dict[f"index_{j}"]

        # calculate the average predicted price
        avg_pred_price[j] = np.average(predicted_prices.loc[index_list])

    print(avg_pred_price)

    # update dataframe with the average predicted price
    df_floors[f"aver_pred_price_{level}"] = df_floors['floors'].map(avg_pred_price)

df_floors.head(10)

{1.0: 435840.47, 1.5: 551543.56, 2.0: 654790.9, 2.5: 1030357.25, 3.0: 590641.56, 3.5: 917995.9}
{1.0: 433019.12, 1.5: 551510.2, 2.0: 648224.56, 2.5: 1029807.6, 3.0: 587794.56, 3.5: 918520.56}
{1.0: 428588.1, 1.5: 550519.44, 2.0: 642811.9, 2.5: 1036431.1, 3.0: 585378.0, 3.5: 934184.8}
{1.0: 426761.34, 1.5: 546365.5, 2.0: 638073.4, 2.5: 1024197.56, 3.0: 579333.0, 3.5: 923300.6}
{1.0: 426126.1, 1.5: 544658.3, 2.0: 636458.6, 2.5: 1016920.44, 3.0: 577503.75, 3.5: 919456.7}
{1.0: 426265.12, 1.5: 544339.3, 2.0: 636406.56, 2.5: 1016434.2, 3.0: 577259.0, 3.5: 919908.5}


,floors,average_price,count,aver_pred_price_1.0,aver_pred_price_1.5,aver_pred_price_2.0,aver_pred_price_2.5,aver_pred_price_3.0,aver_pred_price_3.5
0,1.0,4.421806e+05,10680,4.358405e+05,4.330191e+05,4.285881e+05,4.267613e+05,4.261261e+05,4.262651e+05
1,1.5,5.589806e+05,1910,5.515436e+05,5.515102e+05,5.505194e+05,5.463655e+05,5.446583e+05,5.443393e+05
2,2.0,6.488912e+05,8241,6.547909e+05,6.482246e+05,6.428119e+05,6.380734e+05,6.364586e+05,6.364066e+05
3,2.5,1.060346e+06,161,1.030357e+06,1.029808e+06,1.036431e+06,1.024198e+06,1.016920e+06,1.016434e+06
4,3.0,5.825260e+05,613,5.906416e+05,5.877946e+05,5.853780e+05,5.793330e+05,5.775038e+05,5.772590e+05
5,3.5,9.333125e+05,8,9.179959e+05,9.185206e+05,9.341848e+05,9.233006e+05,9.194567e+05,9.199085e+05


In [ ]:
df_floors.to_csv('floors.csv', index=False)